# [Advent of Code 2021](https://adventofcode.com/2021)

# The toolbox


Generalised pieces of code that either can be used in multiple questions or that simply makes understand the implementation easier.

In [44]:
def Input(day, parser=str.strip, whole_file=False):
    "Fetch the data input from disk."
    filename = f'../data/advent2021/input{day}.txt'
    with open(filename) as fin:
        return mapt(parser, fin)

    
def mapt(fn, *args): 
    "Do a map, and convert the results to a tuple"
    return tuple(map(fn, *args))

# [Day 1](https://adventofcode.com/2021/day/1)

In [50]:
def count_increasing_measurements(scans):
    return sum(
        scan > scans[index - 1]
        for index, scan in enumerate(scans[1:], 1)
    )

In [46]:
data1 = Input(1, int)
count_increasing_measurements(data1)

1121

In [47]:
assert _ == 1121, 'Day 1.1'

In [48]:
chunks = [data1[i:i + 3] for i in range(len(data1) - 2)]
count_increasing_measurements(mapt(sum, chunks))

1065

In [49]:
assert _ == 1065, 'Day 1.2'

## [Day 2](https://adventofcode.com/2021/day/2)

In [57]:
def follow_commands(commands):
    x = d = 0
    for direction, amount in commands:
        if direction == 'forward':
            x += amount
        elif direction == 'down':
            d += amount
        elif direction == 'up':
            d -= amount
        else:
            print('unknown command!')
    return x * d

def parse_input_2(line):
    chunks = line.split(' ')
    return chunks[0], int(chunks[1])

data2 = Input(2, parse_input_2)
follow_commands(data2)

2036120

In [58]:
assert _ == 2036120, "Day 2.1"

In [62]:
def follow_commands(commands):
    x = d = aim = 0
    for direction, amount in commands:
        if direction == 'forward':
            x += amount
            d += aim * amount
        elif direction == 'down':
            aim += amount
        elif direction == 'up':
            aim -= amount
        else:
            print('unknown command!')
    return x * d

follow_commands(data2)

2015547716

In [60]:
assert _ == 2015547716, "Day 2.2"